In [85]:
# import required libraries
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient, Input, Output, load_component
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.entities import BatchEndpoint, PipelineComponentBatchDeployment
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.ai.ml.parallel import parallel_run_function, RunFunction
from dotenv import load_dotenv, find_dotenv
from datetime import datetime
import pandas as pd
import glob
import os

In [4]:
# Initialize ML Client
load_dotenv(find_dotenv(), override=True)

# authenticate
credential = DefaultAzureCredential(tenantid=os.environ.get('TENANT_ID'))

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id = os.environ.get('SUBSCRIPTION_ID'),
    resource_group_name = os.environ.get('RESOURCE_GROUP_NAME'),
    workspace_name = os.environ.get('WORKSPACE_NAME'),
)

print(dict(os.environ))

{'USER': 'zacksoenen', 'VSCODE_WSL_EXT_LOCATION': '/mnt/c/Users/zacksoenen/.vscode/extensions/ms-vscode-remote.remote-wsl-0.81.8', 'WT_PROFILE_ID': '{51855cb2-8cce-5362-8f54-464b92b32386}', 'SHLVL': '2', 'HOME': '/home/zacksoenen', 'OLDPWD': '/mnt/c/Users/zacksoenen/AppData/Local/Programs/Microsoft VS Code', 'DBUS_SESSION_BUS_ADDRESS': 'unix:path=/run/user/1000/bus', 'WSL_DISTRO_NAME': 'Ubuntu', 'WAYLAND_DISPLAY': 'wayland-0', 'LOGNAME': 'zacksoenen', 'PULSE_SERVER': 'unix:/mnt/wslg/PulseServer', 'WSL_INTEROP': '/run/WSL/97485_interop', 'NAME': 'Code', '_': '/home/zacksoenen/miniconda3/envs/mm_remote/bin/python', 'TERM': 'xterm-color', 'PATH': '/home/zacksoenen/miniconda3/envs/mm_remote/bin:/home/zacksoenen/.vscode-server/bin/0ee08df0cf4527e40edc9aa28f4b5bd38bbff2b2/bin/remote-cli:/home/zacksoenen/miniconda3/envs/mm_remote/bin:/home/zacksoenen/miniconda3/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/usr/lib/wsl/lib:/mnt/c/Program Fil

In [6]:
# Use remote environment created in step 1
custom_env_name = "mm-remote-env-py37"

In [7]:
# Acess data asset
data_name = "oj-sim-sales-test"
test_data_asset = ml_client.data.get(data_name, label='latest')
print(test_data_asset.path)

azureml://subscriptions/9a729243-1221-42c5-824c-9e44cb2da98d/resourcegroups/many-models-rg/workspaces/mm-aml-wksp/datastores/workspaceblobstore/paths/LocalUpload/724411e17666e66d19b8612b775b7707/test_subset.csv


In [8]:
# Load data preparation component
partition_data = load_component(source="../src/components/partition_data/partition_data.yaml")

In [9]:
cpu_compute_target = "mm-cpu-cluster"

In [10]:
azureml_tracking_uri = ml_client.workspaces.get(ml_client.workspace_name).mlflow_tracking_uri

In [11]:
'''run_time = datetime.today().strftime('%Y%m%d%H%M%S')

output_dir = f"many_model_outputs/{run_time}/"
print(output_dir)'''

'run_time = datetime.today().strftime(\'%Y%m%d%H%M%S\')\n\noutput_dir = f"many_model_outputs/{run_time}/"\nprint(output_dir)'

In [29]:
# Create parallel inference step
# Declare parallel job, with a run_function task
many_model_inference_with_partition_keys = parallel_run_function(
    name="distributed_inference",
    display_name="Many Model Predictions",
    description="parallel job to batch predict with many models",
    inputs=dict(
        data_source=Input(
            type=AssetTypes.MLTABLE,
            description="Input mltable with predefined partition format.",
            mode=InputOutputModes.DIRECT,  # [Important] To use 'partition_keys', input MLTable is required to use 'direct' mode.
        ),
        drop_cols=Input(
            type="string",
            description="Columns need to be dropped before training. Split by comma.",
        ),
        experiment_names=Input(
            type="string",
            description="Name of training experiement(s) to be used to select best model. Split by comma.",
        ),
        metric_name=Input(
            type="string",
            description="Name of metric to be used to select best model, to be used in mlflow query. i.e. test_remse",
        ),
        date_col=Input(
            type="string",
            description="Name of date column in data",
        ),
        tracking_uri=Input(
            type="string",
            description="tracking uri of mlflow server (aml workspace)",
        ),
    ),
    outputs=dict(
        output_dir=Output(
            type=AssetTypes.URI_FOLDER,
            mode=InputOutputModes.RW_MOUNT,
        )
    ),
    input_data="${{inputs.data_source}}",  # Define which input data will be splitted into mini-batches
    partition_keys=["Store","Brand"],  # Use 'partition_keys' as the data division method. This method requires MLTable input with partition setting pre-defined in MLTable artifact.
    instance_count=6,  # Use X nodes from compute cluster to run this parallel job.
    max_concurrency_per_instance=2,  # Create 2 worker processors in each compute node to execute mini-batches.
    error_threshold=-1,  # Monitor the failures of item processed by the gap between mini-batch input count and returns. 'Many model training' scenario doesn't fit this setting and '-1' means ignore counting failure items by mini-batch returns.
    mini_batch_error_threshold=5,  # Monitor the failed mini-batch by exception, time out, or null return. When failed mini-batch count is higher than this setting, the parallel job will be marked as 'failed'.
    retry_settings=dict(
        max_retries=1,  # Define how many retries when mini-batch execution is failed by exception, time out, or null return.
        timeout=60,  # Define the timeout in second for each mini-batch execution.
    ),
    logging_level="DEBUG", # DEBUG, INFO, WARNING, ERROR, ETC
    task=RunFunction(
        code="../src/",
        entry_script="parallel_inference.py",
        environment=ml_client.environments.get(custom_env_name, label="latest"),
        program_arguments="--drop_cols ${{inputs.drop_cols}} "  # Passthrough input parameters into parallel_train script.
        "--experiment_names ${{inputs.experiment_names}} "
        "--metric_name ${{inputs.metric_name}} "
        "--tracking_uri ${{inputs.tracking_uri}} "
        "--date_col ${{inputs.date_col}} "
        "--output_dir ${{outputs.output_dir}} ",
    ),
)

In [64]:
# Build Pipeline
# Declare the overall input of the job.
test_oj_data = Input(
    path=test_data_asset.path,
    type=AssetTypes.URI_FILE,
    mode=InputOutputModes.RO_MOUNT,
)

# Declare pipeline structure.
@pipeline(display_name="parallel inference output test3")
def parallel_inference_pipeline(pipeline_input_data: Input(type=AssetTypes.MLTABLE)):
    
    # Declare 1st data partition command job.
    partition_job = partition_data(
        data_source=pipeline_input_data,
        partition_keys="Store,Brand",
    )

    # Declare 2nd parallel model training job.
    parallel_inference = many_model_inference_with_partition_keys(
        data_source=partition_job.outputs.tabular_output_data,
        drop_cols="Advert,Store,Brand",
        experiment_names="many-models-parallel-training-job-mlflow-full",
        metric_name="test_rmse",
        date_col="WeekStarting",
        tracking_uri=azureml_tracking_uri
    )
    
    return {"pipeline_output": parallel_inference.outputs.output_dir}

    # User could override parallel job run-level property when invoke that parallel job/component in pipeline.
    # parallel_train.resources.instance_count = 5
    # parallel_train.max_concurrency_per_instance = 2
    # parallel_train.mini_batch_error_threshold = 10

# Create pipeline instance
inference_pipeline = parallel_inference_pipeline(pipeline_input_data=test_oj_data,)

# Set pipeline level compute
inference_pipeline.settings.default_compute = cpu_compute_target
print(inference_pipeline)

display_name: parallel inference output test3
type: pipeline
inputs:
  pipeline_input_data:
    mode: ro_mount
    type: uri_file
    path: azureml://subscriptions/9a729243-1221-42c5-824c-9e44cb2da98d/resourcegroups/many-models-rg/workspaces/mm-aml-wksp/datastores/workspaceblobstore/paths/LocalUpload/724411e17666e66d19b8612b775b7707/test_subset.csv
outputs:
  pipeline_output:
    mode: rw_mount
    type: uri_folder
jobs:
  partition_job:
    type: command
    inputs:
      data_source:
        path: ${{parent.inputs.pipeline_input_data}}
      partition_keys: Store,Brand
    component:
      $schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
      name: partition_data
      version: '1'
      display_name: Partition data by keys
      type: command
      inputs:
        data_source:
          type: uri_file
        partition_keys:
          type: string
      outputs:
        tabular_output_data:
          type: mltable
      command: python partition_dat

In [65]:
# Run Inference Pipeline
pipeline_job = ml_client.jobs.create_or_update(
    inference_pipeline,
    experiment_name="many-models-parallel-inference-job",
)

### Analyze Outputs ###

In [86]:
# Download output data
local_download_path = "../data/oj_sim_sales/outputs/"
ml_client.jobs.download(name=pipeline_job.name, download_path=local_download_path, output_name="pipeline_output")

In [88]:
# Read output data as pandas df

glob_path = os.path.join(local_download_path +"named-outputs/pipeline_output/*.csv")
output_files = glob.glob(glob_path)
output_df = pd.concat((pd.read_csv(f) for f in output_files))

display(len(output_df))
display(output_df.head())

1386

,WeekStarting,Advert,Price,Store,Brand,predictions
0,1992-07-02,True,2.55,1028,minute_maid,35169.553746
1,1992-07-09,True,2.07,1028,minute_maid,31636.904183
2,1992-07-16,True,2.04,1028,minute_maid,31391.641231
3,1992-07-23,True,2.17,1028,minute_maid,28728.226342
4,1992-07-30,True,2.16,1028,minute_maid,24841.971246


### Create and Deploy Batch Endpoint

In [53]:
# Create Batch Endpoint

endpoint_name = "manymodels-batch-endp-demo"
print(f"Endpoint name: {endpoint_name}")

endpoint = BatchEndpoint(
    name=endpoint_name,
    description="A many models batch inference endpoint",
)

ml_client.batch_endpoints.begin_create_or_update(endpoint).result()

Endpoint name: manymodels-batch-endp-demo


BatchEndpoint({'scoring_uri': 'https://manymodels-batch-endp-demo.centralus.inference.ml.azure.com/jobs', 'openapi_uri': None, 'provisioning_state': 'Succeeded', 'name': 'manymodels-batch-endp-demo', 'description': 'A many models batch inference endpoint', 'tags': {}, 'properties': {'BatchEndpointCreationApiVersion': '2022-05-01', 'azureml.onlineendpointid': '/subscriptions/9a729243-1221-42c5-824c-9e44cb2da98d/resourceGroups/many-models-rg/providers/Microsoft.MachineLearningServices/workspaces/mm-aml-wksp/batchEndpoints/manymodels-batch-endp-demo'}, 'print_as_yaml': True, 'id': '/subscriptions/9a729243-1221-42c5-824c-9e44cb2da98d/resourceGroups/many-models-rg/providers/Microsoft.MachineLearningServices/workspaces/mm-aml-wksp/batchEndpoints/manymodels-batch-endp-demo', 'Resource__source_path': None, 'base_path': '/home/zacksoenen/Projects/many-models-azureml/demo_notebooks', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7fe62c3a4d10>, 'auth_mode': '

In [66]:
# Define Pipeline Component
pipeline_component = parallel_inference_pipeline().component

# Register Pipeline Component for better tracking and versioning
ml_client.components.create_or_update(pipeline_component)

PipelineComponent({'intellectual_property': None, 'auto_increment_version': False, 'source': 'REMOTE.WORKSPACE.COMPONENT', 'is_anonymous': False, 'auto_delete_setting': None, 'name': 'parallel_inference_pipeline', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': '/subscriptions/9a729243-1221-42c5-824c-9e44cb2da98d/resourceGroups/many-models-rg/providers/Microsoft.MachineLearningServices/workspaces/mm-aml-wksp/components/parallel_inference_pipeline/versions/1', 'Resource__source_path': None, 'base_path': '/home/zacksoenen/Projects/many-models-azureml/demo_notebooks', 'creation_context': <azure.ai.ml._restclient.v2022_10_01.models._models_py3.SystemData object at 0x7fe532394910>, 'serialize': <msrest.serialization.Serializer object at 0x7fe61c838650>, 'version': '1', 'latest_version': None, 'schema': None, 'type': 'pipeline', 'display_name': 'parallel inference output test3', 'is_deterministic': False, 'inputs': {'pipeline_input_data': {'type': 'mltable', '

In [69]:
# Create Deployment
deployment = PipelineComponentBatchDeployment(
    name="many-models-inference-deployment",
    description="A many models deployment.",
    endpoint_name=endpoint_name,
    component=pipeline_component,
    settings={"default_compute": cpu_compute_target},
)

ml_client.batch_deployments.begin_create_or_update(deployment).result()

Class PipelineComponentBatchDeployment: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


PipelineComponentBatchDeployment({'type': None, 'name': 'many-models-inference-deployment', 'description': None, 'tags': {'PipelineDeployment.ComponentId': '/subscriptions/9a729243-1221-42c5-824c-9e44cb2da98d/resourceGroups/many-models-rg/providers/Microsoft.MachineLearningServices/workspaces/mm-aml-wksp/components/parallel_inference_pipeline/labels/default'}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': '/home/zacksoenen/Projects/many-models-azureml/demo_notebooks', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7fe62c247ad0>, 'component': '/subscriptions/9a729243-1221-42c5-824c-9e44cb2da98d/resourceGroups/many-models-rg/providers/Microsoft.MachineLearningServices/workspaces/mm-aml-wksp/components/parallel_inference_pipeline/labels/default', 'endpoint_name': 'manymodels-batch-endp-demo', 'settings': None, 'job_definition': None})

In [91]:
# Invoke Batch endpoint
input_data = test_oj_data

endp_job = ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint.name,
    deployment_name=deployment.name,
    inputs={"pipeline_input_data": input_data},
)

ml_client.jobs.get(endp_job.name)

Experiment,Name,Type,Status,Details Page
manymodels-batch-endp-demo,5eefd257-6f55-4240-acda-c2bfb1fc1781,pipeline,Failed,Link to Azure Machine Learning studio
